# Case Classification with BERT(Train New Model)

Applying BERT to the problem of multiclass text classification. Our dataset consists of messages. Each dialog utterance/message is labeled with one of the two emotion categories: normal or attenion. 

## Workflow: 
1. Import Data
2. Data preprocessing and downloading BERT
3. Training and validation
4. Saving the model


👋  **Let's start** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 


Mounted at /content/drive


In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain
!pip3 install sklearn

     |████████████████████████████████| 25.3MB 124kB/s 
     |████████████████████████████████| 6.8MB 49.9MB/s 
     |████████████████████████████████| 983kB 45.1MB/s 
     |████████████████████████████████| 266kB 53.4MB/s 
     |████████████████████████████████| 1.9MB 38.7MB/s 
     |████████████████████████████████| 1.2MB 27.5MB/s 
     |████████████████████████████████| 471kB 48.7MB/s 
     |████████████████████████████████| 901kB 42.2MB/s 
     |████████████████████████████████| 3.3MB 45.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=bfe77c7ffad3a71cc2d41c49beebbb80c1b03e2e55b03f714a6b7b70ef5197d9
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=b5064ff4b5d3105ec9e6ff84cb4139eb734e15c7c6e4f8eadb07d7899d2149d9
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

##Train using BERT

In [ ]:
data_train = pd.read_csv("//content//drive//My Drive//Data/nlpdata//gen_data//data_train.csv", encoding='utf-8')
data_test = pd.read_csv("//content//drive//My Drive//Data/nlpdata//gen_data/data_test.csv", encoding='utf-8')


X_train = data_train.case.tolist()
X_test = data_test.case.tolist()

y_train = data_train.label.tolist()
y_test = data_test.label.tolist()


In [ ]:
encoding = {
    'normal': 0,
    'attention': 1,
}
# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]


In [ ]:
class_names = ['normal', 'attention']
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=500, 
                                                                       max_features=40000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=4)

In [ ]:
learner.fit_onecycle(5e-5, 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

# predictor.save("//content//drive//My Drive//Data//nlpdata//newmodel//")



begin training using onecycle policy with max lr of 5e-05...
88/88 [==============================] - 90s 800ms/step - loss: 0.5071 - accuracy: 0.7714 - val_loss: 0.2740 - val_accuracy: 0.8864


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
predictor.save("//content//drive//My Drive//Data//nlpdata//newmodel//")

In [ ]:
import ktrain


# reload the predictor
predictor = ktrain.load_predictor("//content//drive//My Drive//Data//nlpdata//newmodel//")



In [ ]:
import time 
class_names = ['normal', 'attention']
# message = 'Incident Location: 783C XXX RISE #13-09  XXX3783) if the problem cannot resolve it is very dangerous to the residents , i have report  3 month ago' #attention
# message="Incident Location: 716 XXX DR 70 #10-136  XXX0716) Incident Location Description:  Mr Loke enquire how long can a unit do renovation, his neighbor below #09-136 started before CNY and drilling noise can still be heard now. He wish to indicate that the renovation noise is not from his unit, one of his neighbor approached his unit to check. He enquire if anyone else feedback and when will it stop. Please assist, thank you.."
# message="The window grill is loose not able to fasten, it can drop downstair. It is very dangerous for my child at home. Please assist me asap " # attention
# message=" Request make many times to the office but no one reponse to me. I call to office noone answer!" #attention
# message="Ceiling leaking, need help to repair. Reported"#attenion
# message="Ceiling leaking, need help to repair. Resend"#attenion
# message="Incident Location: 720 XXX AVE 6 #12-616  XXX0720) Incident Location Description:  Feedback the clg leaks at master toilet.  Refer to EE for assistance and return call. thanks" #normal
# message="Incident Location: 727 XXX CIRCLE #06-106  XXX0727) Incident Location Description:  Onwer had book eappt on 10/3 at 8.30am with regards his ceiling leaking at the kitchen toilet. Request HDB officer to check and rectified the issue as soon as possible before the leaking become more worse. Inform him that with route his feedback to the EEIC to follow up with him and he does not need to come down to BO. He noted. EEic for your follow up with owner." #normal
# message=" Request make more two week. No feedback from office!" #attention
# message="There are persistence noise of renovation affect my work. Call the office to report case since 2 weeks ago. There is an urgent response required" #attention
# message="There are burning smoke around my block for a few week. I have report 2 weeks ago . Please help to assist to investigate." #attention
# message="Put up request for repair more than 3 weeks but no feedback" #attention
# message="Feedback on defects. Thanks.   1. toilet door handle broke when the door slammed due to the strong wind today. The handle was already loose before it broke.  2. The toilet tap spoilt."#normal 
# message="there are many crack line at the ceiling. I need you urgent attention!" #attention
# message="Incident Location: 783C XXX RISE #13-09  XXX3783) Incident Location Description:  From: Bei Er ONG (HDB)  ONG_Bei_Er_hdb.gov.sg  Sent: Wednesday, 7 April 2021 12:40 PM To: Marpuah KAWI (HDB)  Marpuah_KAWI_hdb.gov.sg Cc: Angelia MH LIM (HDB)  Angelia_MH_LIM_hdb.gov.sg Subject: FW: Master bedroom ceiling light water leakage HDB ref. no.: 91524919216 Address: BLK 783C XXX RISE # 13 - 09 XXX3783)Hi Marpuah Please create CMS for Angelia. Thanks! Regards Bei Er Ong Bei "
# message="I totally disappoint with the services." # attention
# message="I am unhappy about the repair done in my ceiling. Not up to standard! "
# message=" The water pipe near my unit walkway is leaking. Please help." #normal
# message="feedback of defectives 3 weeks ago "
# message="Incident Location: 783C XXX RISE #13-09  XXX3783) his a repeat case.Incident Location Description:  From: Bei Er ONG (HDB)  ONG_Bei_Er_hdb.gov.sg  Sent: Wednesday, 7 April 2021 12:40 PM To: Marpuah KAWI (HDB)  Marpuah_KAWI_hdb.gov.sg Cc: Angelia MH LIM (HDB)  Angelia_MH_LIM_hdb.gov.sg Subject: FW: Master bedroom ceiling light water leakage HDB ref. no.: 91524919216 Address: BLK 783C XXX RISE # 13 - 09 XXX3783).This a repeat case.Hi Marpuah Please create CMS for Angelia .Thanks! Regards Bei Er Ong Bei "
# message="pls help urgent"
start_time = time.time() 
print(predictor.get_classes())
prediction = predictor.predict(message,return_proba=True)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))
print('predicted: {} ({:.2f} seconds)'.format(class_names[np.argmax(prediction)], (time.time() - start_time)))

['normal', 'attention']
predicted: [0.794546   0.20545399] (0.12)
predicted: normal (0.12 seconds)
